This notebook enables to train TiSSNet model.

In [ ]:
import random
import csv

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

from utils.training.metrics import accuracy_for_segmenter, AUC_for_segmenter
from utils.training.data_loading import lines_to_line_generator, get_line_to_spectro_seg
from utils.training.keras_models import TiSSNet

## Parameters

In [ ]:
ROOT_DIR = "/path/to/the/dataset"  # path where we expect to find directories named "postives", "negatives" and a csv file
SEED = 0  # seed used for RND (shuffling)
BATCH_SIZE = 64
EPOCHS = 50
CHECKPOINTS_DIR = "../../../data/model_saves/TiSSNet"  # directory where the model will save its history and checkpoints

FOLDS = 5  # number of folds for the cross-validation
SIZE = (128, 186)  # number of pixels in the spectrograms
DURATION_S = 100  # duration of the spectrograms in s
OBJECTIVE_CURVE_WIDTH = 10  # defines width of objective function in s

data_loader = get_line_to_spectro_seg(size=SIZE, duration_s=DURATION_S, channels=1, objective_curve_width=OBJECTIVE_CURVE_WIDTH)
model = TiSSNet

## Load data

In [ ]:
# list the data samples
with open(ROOT_DIR + "/dataset.csv", "r") as f:
    csv_reader = csv.reader(f, delimiter=",")
    lines = list(csv_reader)
print(len(lines), "files found")

pos = [l for l in lines if l[1]=="positive"]
neg = [l for l in lines if l[1]=="negative"]
random.Random(SEED).shuffle(pos)
random.Random(SEED).shuffle(neg)
print(f"{len(pos)} positive files found and {len(neg)} negative files found")

# Make the datasets for a cross-validation approach
train_datasets, valid_datasets = [], []
for i in range(FOLDS):
    start_valid_idx = int(len(pos) * i / FOLDS)
    end_valid_idx = int(len(pos) * (i + 1) / FOLDS)
    # unbalanced training set
    train_files = pos[:start_valid_idx]+pos[end_valid_idx:]+neg[:start_valid_idx]+neg[end_valid_idx:] 
    # balanced validation set
    valid_files = pos[start_valid_idx:end_valid_idx]+neg[start_valid_idx:end_valid_idx] 
    random.Random(SEED).shuffle(train_files)
    random.Random(SEED).shuffle(valid_files)
    train_generator = lines_to_line_generator(train_files)
    valid_generator = lines_to_line_generator(valid_files)
    train_datasets.append(tf.data.Dataset.from_generator(lambda: map(tuple, train_generator), output_signature=tf.TensorSpec(shape=[None], dtype=tf.string)))
    valid_datasets.append(tf.data.Dataset.from_generator(lambda: map(tuple, valid_generator), output_signature=tf.TensorSpec(shape=[None], dtype=tf.string)))
    
    train_datasets[-1] = train_datasets[-1].map(data_loader).batch(batch_size=BATCH_SIZE)
    valid_datasets[-1] = valid_datasets[-1].map(data_loader).batch(batch_size=BATCH_SIZE)

# make a dataset with all the data, for the training one cross-validation is done
all_train_files = pos + neg
random.Random(SEED).shuffle(all_train_files)
all_train_generator = lines_to_line_generator(all_train_files)
all_train_dataset =tf.data.Dataset.from_generator(lambda: map(tuple, all_train_generator), output_signature=tf.TensorSpec(shape=[None], dtype=tf.string))
all_train_dataset = all_train_dataset.map(data_loader).batch(batch_size=BATCH_SIZE)

## Plot of the data

In [ ]:
cols = 8
lines = 2
batch_number = 1  # number of the batch we want to inspect

to_show = cols * lines
plt.figure(figsize=(cols*2.5, lines*5))
shown=0
for images, y in valid_datasets[0].take(batch_number+to_show//BATCH_SIZE+1):
    if batch_number:
            batch_number -= 1
            continue
    for i in range(min(BATCH_SIZE, to_show-shown)):
        ax1 = plt.subplot(lines*2, cols, 1 + shown%cols + cols*2*(shown//cols))
        plt.xlabel("time")
        plt.ylabel("frequency")
        plt.imshow(images[i].numpy(), cmap='inferno')

        ax2 = plt.subplot(lines*2, cols, 1 + shown%cols + cols*2*(shown//cols) + cols)

        ax2.plot(y[i], label='ground truth')
        ax2.legend(loc="upper left")
        ax2.set_xlim([0, SIZE[1]])
        ax2.set_ylim([0, 1])
        ax2.set_xlabel("time")
        ax2.set_ylabel("probability")

        shown += 1
plt.show()

## Cross-validation training

In [ ]:
import os, pickle

for i in range(FOLDS):
    path_prefix = f'{CHECKPOINTS_DIR}/FOLD-{i}'
    history_file = f'{path_prefix}/history.pkl'
    
    if os.path.isfile(history_file):
        print(f"fold {i} already has an history file, skipping it")
        continue
        
    print(f"starting training of fold {i}")
    m = model()

    m.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss=tf.losses.binary_crossentropy,
            metrics=[accuracy_for_segmenter, AUC_for_segmenter()])

    m.build((BATCH_SIZE, SIZE[0], SIZE[1], 1))

    if i==0:
        m.summary()

    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=f'{path_prefix}/cp-{{epoch:04d}}.ckpt', save_weights_only=True, verbose=1)

    history = m.fit(
            train_datasets[i],
            batch_size=BATCH_SIZE,
            validation_data=valid_datasets[i],
            steps_per_epoch=len(train_files) // BATCH_SIZE,
            validation_steps=len(valid_files) // BATCH_SIZE,
            epochs=EPOCHS,
            callbacks=[cp_callback]
        )
    
    with open(history_file, 'wb') as f:
        pickle.dump(history.history, f)

## Training on all the data

In [ ]:
m = model()

m.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss=tf.losses.binary_crossentropy,
        metrics=accuracy_for_segmenter)

m.build((BATCH_SIZE, SIZE[0], SIZE[1], 1))

m.summary()

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=f"{CHECKPOINTS_DIR}/checkpoints/cp-{{epoch:04d}}.ckpt",
                                                     save_weights_only=True,
                                                     verbose=1)

m.fit(
        all_train_dataset,
        batch_size=BATCH_SIZE,
        steps_per_epoch=(len(train_files) + len(valid_files))// BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[cp_callback]
    )

## Plot some examples of outputs of the network

In [ ]:
m = model()
epoch = 22  # epoch checkpoint that we want to load
m.load_weights(f"{CHECKPOINTS_DIR}/checkpoints/cp-{{epoch:04d}}.ckpt")

In [ ]:
cols = 5
lines = 10
batch_number = 0

to_show = cols * lines
plt.figure(figsize=(cols*5, lines*10))
shown=0
for images, y in valid_datasets[0].take(batch_number+to_show//BATCH_SIZE+1):
    if batch_number:
            batch_number -= 1
            continue
    for i in range(min(BATCH_SIZE, to_show-shown)):
        ax1 = plt.subplot(lines*2, cols, 1 + i%cols + cols*2*(i//cols))
        plt.xlabel("time (s)")
        plt.ylabel("frequency (Hz)")
        plt.imshow(images[i].numpy(), cmap='inferno', extent=(0, 100, 0, 120))

        ax2 = plt.subplot(lines*2, cols, 1 + i%cols + cols*2*(i//cols) + cols)
        
        #ax2.plot(y[i], label='ground truth')
        predicted = m.predict(np.reshape(images[i], (1, SIZE[0], SIZE[1], 1)), verbose=False)[0]
        ax2.plot(predicted, label='predicted')
        ax2.legend(loc="upper left")
        ax2.set_xlim([0, SIZE[1]])
        ax2.set_ylim([0, 1])
        ax2.set_xlabel("time (s)")
        ax2.set_ylabel("probability")

        shown += 1
plt.show()